In [1]:
import pandas as pd
import pickle
import json
import os
import google.oauth2.credentials
import googleapiclient.discovery

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

 
# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"
 
# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [2]:
def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()
 
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)
 
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [3]:
def write_csv(comments):
    df = pd.DataFrame(comments)
    df.columns = ["Comments"]
    print(df)
    df.to_csv("Comments.csv", encoding='utf-8', index=False)

In [4]:
def get_video_id(service, url,limit):
    id = url.split("v=")[1]
    video_id = id.split("&")[0]
    
    get_video_comments(service,limit, part="snippet",
                                maxResults=100,
                                order="time",
                                textFormat="plainText",
                                videoId=video_id)

In [17]:
def get_video_comments(service, limit, **kwargs):
    comments = set()
    
    request = service.commentThreads().list(**kwargs)
    response = request.execute()
    try:
        while(response["nextPageToken"]):
            print(response["nextPageToken"])

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                comments.add(comment)
                if(len(comments)>=limit):
                    return write_csv(comments)
            try:
                request = service.commentThreads().list(**kwargs,pageToken=response["nextPageToken"])
                response = request.execute()
            except HttpError:
                os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
                service = get_authenticated_service()
            
    except KeyError or KeyboardInterrupt:
        return write_csv(comments)

In [ ]:
if __name__ == "__main__":
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.

    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()

#     url = input("Enter video link: ")
    url = "https://www.youtube.com/watch?v=uoxtAXN2tnI"
    
    max_comments = int(input("How many comments approximately do you want ? (NUMBER): "))

    get_video_id(service,url,max_comments)